In [ ]:
import h5py
import numpy as np
import imageio
from cellpose import models
from cellpose import utils as cp_utils
from cellstitch.pipeline import *
from cellstitch.evaluation import *
from cellstitch.utils import *
import pandas as pd

In [ ]:
dataset = "ovules_subsampled" # or ovules_subsampled

# Pipeline Benchmark

Comparision between cellstitch (2D), cellpose3D (2.5D), plantseg (3D), cellstitch3D; using the same training set.

### PlantSeg
- First, created a plantseg virtual enviroment: 
`conda create -n plant-seg -c pytorch -c conda-forge -c lcerrone -c awolny pytorch=1.9 pytorch-3dunet=1.3.7 plantseg` 
- activate the environment: `conda activate plant-seg` 
- download the ovules test dataset: https://osf.io/uzq3w/ to `../data/ovules/plantseg_test/` 
- set the `path` in `plantseg.yaml` to `../data/ovules/plantseg_test/` 
- perform segmentation with the `confocal_unet_bce_dice_ds1x` by running `plantseg --config plantseg.yaml`
- the predictions are saved to `../data/ovules/plantseg_test/PreProcessing/confocal_unet_bce_dice_ds1x/
`

In [ ]:
test_filenames = ["N_294_final_crop_ds2.npy", 
                 "N_435_final_crop_ds2.npy",
                 "N_441_final_crop_ds2.npy",
                 "N_511_final_crop_ds2.npy",
                 "N_522_final_crop_ds2.npy",
                 "N_590_final_crop_ds2.npy",
                 "N_593_final_crop_ds2.npy"]

In [ ]:
plantseg_results_folder = "../data/ovules/plantseg_test/PreProcessing/confocal_unet_bce_dice_ds1x/MultiCut/"

In [ ]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename) 
    
    with h5py.File("%s/%s_predictions_multicut.h5" % (plantseg_results_folder, test_filename), "r") as f:
        plantseg = np.array(list(f['segmentation'])) 
        
    plantseg[np.where(plantseg == 1)] = 0 # plantseg use 1 as labels
    np.save("./results/ovules/pipeline/plantseg/%s.npy" % test_filename, plantseg)

### Train cellpose model from scratch
First, need to prepare training data for cellpose.

In [ ]:
train_filenames = ["N_404_ds2x.npy", 
                  "N_405_A_ds2x.npy", 
                  "N_405_B_ds2x.npy", 
                  "N_416_ds2x.npy",
                  "N_422_ds2x.npy",
                  "N_425_ds2x.npy",
                  "N_428_ds2x.npy",
                  "N_440_ds2x.npy",
                  "N_445_ds2x.npy",
                  "N_449_ds2x.npy",
                  "N_450_ds2x.npy", 
                  "N_451_ds2x.npy",
                  "N_454_ds2x.npy",
                  "N_457_ds2x.npy",
                  "N_458_ds2x.npy",
                  "N_487_ds2x.npy",
                  "N_509_ds2x.npy",
                  "N_512_ds2x.npy",
                   "N_517_ds2x.npy",
                  "N_534_ds2x.npy",
                  "N_535_ds2x.npy",
                  "N_536_ds2x.npy"]

ovules_folder = "../DATA/ovules"
cellpose_folder = "../DATA/ovules/cellpose_train"

In [ ]:
for train_filename in train_filenames: 
    img = np.load("%s/raw/%s" % (ovules_folder, train_filename))
    labels = np.load("%s/labels/%s" % (ovules_folder, train_filename)) 
    depth = img.shape[0] 
    
    for i in range(depth): 
        imageio.imwrite("%s/%s_%s.tif" % (cellpose_folder, train_filename, i), img[i])
        imageio.imwrite("%s/%s_%s_masks.tif" % (cellpose_folder, train_filename, i), labels[i])

`python -m cellpose --train --dir ../DATA/ovules/cellpose_train --pretrained_model None --n_epochs 100  --verbose` 

### Generate cellpose3d results

In [ ]:
model_dir = '../DATA/ovules/cellpose_train/models/cellpose_residual_on_style_on_concatenation_off_cellpose_train_2023_05_08_09_31_06.231473'

In [ ]:
flow_threshold = 1
model = models.CellposeModel(gpu=True, pretrained_model=model_dir)

In [ ]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename)
    img = np.load("%s/raw/%s" % (ovules_folder, test_filename)) 
    masks, _, _ = model.eval(img, do_3D=True, flow_threshold=flow_threshold, channels = [0,0]) 
    np.save("./results/ovules/pipeline/cellpose3d/%s" % test_filename, masks) 

### Generate cellpose2d results

In [ ]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename)
    img = np.load("../DATA/%s/raw/%s" % (dataset, test_filename)) 
    masks, _, _ = model.eval(list(img), do_3D=False, flow_threshold=flow_threshold, channels = [0,0])
    masks = cp_utils.stitch3D(np.array(masks))
    
    np.save("./results/%s/cellpose2d/%s" % (dataset, test_filename), masks)

### Generate cellstitch results

In [ ]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename)
    img = np.load("%s/raw/%s" % (ovules_folder, test_filename)) 
    
    cellstitch, _, _ = model.eval(list(img), flow_threshold=flow_threshold, channels = [0,0])
    cellstitch = np.array(cellstitch)

    yz_masks, _, _ = model.eval(list(img.transpose(1,0,2)), flow_threshold=flow_threshold, channels = [0,0])
    yz_masks = np.array(yz_masks).transpose(1,0,2)

    xz_masks, _, _ = model.eval(list(img.transpose(2,1,0)), flow_threshold=flow_threshold, channels = [0,0])
    xz_masks = np.array(xz_masks).transpose(2,1,0)

    full_stitch(cellstitch, yz_masks, xz_masks)
    
    np.save("./results/ovules/pipeline/cellstitch/%s" % test_filename, cellstitch) 

# benchmark results

In [ ]:
method = "cellpose2d" 
ap_threshold = 0.5

data = [] 
for filename in test_filenames:
    print("Starting %s" % filename)
    labels = np.load('../DATA/%s/labels/%s' % (dataset, filename)) 
    
    masks = np.load("./results/%s/%s/%s" % (dataset, method, filename))

    ap, tp, fp, fn = average_precision(labels, masks, ap_threshold)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    
    row = [ 
        filename, 
        ap, 
        tp, 
        fp, 
        fn, 
        precision,
        recall
    ]
    
    data.append(row)

df = pd.DataFrame(data, columns=[
    "filename",
    "ap", 
    "tp", 
    "fp", 
    "fn",
    "precision",
    "recall"
])

df.to_csv("./results/%s/%s.csv" % (dataset, method), index=False)